In [1]:
import glob
import pandas
import numpy

import matplotlib
import matplotlib.pyplot as plot

from plots import plot_boxes

ImportError: cannot import name cbook

In [7]:
pandas.__version__

u'0.24.1'

In [12]:
target_dir = 'results/'

data_load = None
for file_name in glob.glob(target_dir + '*'):
    print('Loading...\t' + file_name)
    if data_load is None:
        data_load = pandas.read_pickle(file_name)
    else:
        data_load = pandas.concat([data_load,
                                   pandas.read_pickle(file_name)], 
                                  sort=False)

Loading...	results/gaussian_covariance_100_20000_20190228
Loading...	results/gaussian_covariance_100_350000_20190228
Loading...	results/gaussian_itentity_20000_100_20190228
Loading...	results/gaussian_itentity_350000_100_20190228
Loading...	results/gaussian_covariance_100_100000_20190228
Loading...	results/gaussian_itentity_100000_100_20190228


In [13]:
print(data_load['mechanism'].value_counts())
print(data_load.columns.values)
data_load[data_load.mechanism == 'baseline']


gaussian    600
Name: mechanism, dtype: int64
['mechanism' 'query' 'sample size' 'iteration' 'metric' 'result'
 'mechanism runtime (s)' 'total runtime (s)']


,mechanism,query,sample size,iteration,metric,result,mechanism runtime (s),total runtime (s)


# Compare Differentially Private (DP) Sample Covarince / SVD methods

In [15]:
sample_cov_data = data_load[data_load.metric == 'principle component RSS']

In [21]:
line_data = None

# pandas.concat([
#     sample_cov_data[ sample_cov_data['query'] == 'indentity'],
#     sample_cov_data[ sample_cov_data['query'] == 'covariance']    
# ], axis=1)
plot_fields = ['mechanism', 'sample size', 'query', 'result', 'metric']

result = (sample_cov_data[plot_fields]
          .groupby(by=['mechanism', 'sample size', 'query'])
          .median()
          .unstack())


# plot.plot('sample_size', )

In [22]:
fig, ax = plot.subplots(figsize=(15,7))

sample_cov_data[plot_fields]
          .groupby(by=['mechanism', 'sample size', 'query'])
          .median()
          .unstack()
          .plot(ax=ax)
        


IndentationError: unexpected indent (<ipython-input-22-a89fd900e272>, line 4)

In [23]:
(sample_cov_data[plot_fields]
 .groupby(by=['mechanism', 'sample size', 'query'])
 .quantile([0.05,0.95])
 .unstack())

result          
                                      0.05      0.95
mechanism sample size query                         
gaussian  20000       covariance  0.043536  0.114288
                      itentity    0.378029  0.411264
          100000      covariance  0.001533  0.007907
                      itentity    0.384562  0.409987
          350000      covariance  0.000115  0.000486
                      itentity    0.382758  0.411493

In [ ]:
# Collect data and labels
boxplot_labels = [ sample_size for sample_size in sample_cov_data['sample size'].unique()]
boxplot_data = [ sample_cov_data[sample_cov_data['sample size'] == sample_size].value_counts() ] 
                for sample_size in sample_cov_data['sample size'].unique()]

In [ ]:
# Boxplot of experiment RMSE results 
boxplot_args = dict(notch=False,bootstrap=1000)

plot, boxplot = plot_boxes(
        title='Comparing Captured Variance of DP\nSample Covariance Methods',
        xlabel='Sample Size',
        ylabel='Principal Component',
        xplot_labels=e1_rmse_labels,
        plot_data=e1_rmse_data,
        xplot_label_args=dict(rotation=45, fontsize=8),
        **boxplot_args)